<a href="https://colab.research.google.com/github/UIT-21520916/NCKH/blob/task4/task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import numpy as np
import pandas as pd
import glob
import time
import pandas as pd
# from xml.dom import minidom


import os
import matplotlib.pyplot as plt
import keras

import cv2

In [4]:
df=pd.read_csv('/home/XSS_dataset.csv', encoding='utf-8-sig')
# with open('/home/XSS_dataset.csv') as df:
#   df = pd.read_csv(df, names=header_list)

In [5]:
df.head()

,Unnamed: 0,Sentence,Label
0,0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,2,"\t </span> <span class=""reference-text"">Steeri...",0
3,3,"\t </span> <span class=""reference-text""><cite ...",0
4,4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [6]:
df=df[df.columns[-2:]]

In [7]:
df.head()

,Sentence,Label
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,"\t </span> <span class=""reference-text"">Steeri...",0
3,"\t </span> <span class=""reference-text""><cite ...",0
4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [8]:
df.shape
df.info()
print(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13686 entries, 0 to 13685
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  13686 non-null  object
 1   Label     13686 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 214.0+ KB
                                                Sentence  Label
0      <li><a href="/wiki/File:Socrates.png" class="i...      0
1                   <tt onmouseover="alert(1)">test</tt>      1
2      \t </span> <span class="reference-text">Steeri...      0
3      \t </span> <span class="reference-text"><cite ...      0
4      \t </span>. <a href="/wiki/Digital_object_iden...      0
...                                                  ...    ...
13681             <img onpointerenter=alert(1)>XSS</img>      1
13682  <source onbeforepaste="alert(1)" contenteditab...      1
13683  <div draggable="true" contenteditable>drag me<...      1
13684  <li><cite id="CITEREFDomingos2015" class=

In [9]:
# Convert to ASCII

def convert_to_ascii(sentence):
    sentence_ascii=[]

    for i in sentence:
        
        """Some characters have values very big e.d 8221 adn some are chinese letters
        I am removing letters having values greater than 8222 and for rest greater 
        than 128 and smaller than 8222 assigning them values so they can easily be normalized"""
       
        if(ord(i)<8222):      # ” has ASCII of 8221
            
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(134)
            
            
            if(ord(i)==8221): # ”  :  8221
                sentence_ascii.append(129)
                
            if(ord(i)==8220): # “  :  8220
                sentence_ascii.append(130)
                
                
            if(ord(i)==8216): # ‘  :  8216
                sentence_ascii.append(131)
                
            if(ord(i)==8217): # ’  :  8217
                sentence_ascii.append(132)
            
            if(ord(i)==8211): # –  :  8211
                sentence_ascii.append(133)
                
                
            """
            If values less than 128 store them else discard them
            """
            if (ord(i)<=128):
                    sentence_ascii.append(ord(i))
    
            else:
                    pass
            

    zer=np.zeros((10000))

    for i in range(len(sentence_ascii)):
        zer[i]=sentence_ascii[i]

    zer.shape=(100, 100)


#     plt.plot(image)
#     plt.show()
    return zer


In [33]:


# send each sentence to be converted to ASCII


arr=np.zeros((df.shape[0],100,100)) # 1 mảng gồm len(sentences) mảng (100x100)
arr1=np.zeros((df.shape[0],100,100))
for i in range(df.shape[0]):
    
    image=convert_to_ascii(df['Sentence'][i]) 

    # arr1[i]=np.array(image,dtype='float')

    x=np.asarray(image,dtype='float') #khi mà image thay đổi thì x cũng thay đổi theo
    image =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
    # tại sao phải resize khi mà dsize không thay đổi


    image/=128

    
#     if i==1:
#         plt.plot(image)
#         plt.show()    
    arr[i]=image



In [34]:
print(arr[3])
# print(arr1[3]) 

[[0.0703125 0.25      0.46875   ... 0.7578125 0.84375   0.25     ]
 [0.90625   0.7890625 0.9375    ... 0.90625   0.9140625 0.890625 ]
 [0.7890625 0.25      0.6171875 ... 0.78125   0.9140625 0.7734375]
 ...
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]
 [0.        0.        0.        ... 0.        0.        0.       ]]
[[  9.  32.  60. ...  97. 108.  32.]
 [116. 101. 120. ... 116. 117. 114.]
 [101.  32.  79. ... 100. 117.  99.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]


In [35]:
print("Input data shape : ", arr.shape)

Input data shape :  (13686, 100, 100)


In [36]:
# Reshape data for input to CNN
data = arr.reshape(arr.shape[0], 100, 100, 1)

In [37]:
data.shape

(13686, 100, 100, 1)

In [38]:
y=df['Label'].values